Advanced: Alternate Backends
============================

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.4 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
#!pip install -I "phoebe>=2.4,<2.5"

In [2]:
import phoebe
from phoebe import u # units

logger = phoebe.logger()

b = phoebe.default_binary()

And we'll attach some dummy datasets.  See [Datasets](datasets.html) for more details.

In [3]:
b.add_dataset('orb', 
              compute_times=phoebe.linspace(0,10,1000), 
              dataset='orb01')

<ParameterSet: 50 parameters | contexts: dataset, figure, compute, constraint>

In [4]:
b.add_dataset('lc', 
              compute_times=phoebe.linspace(0,10,1000), 
              dataset='lc01')

<ParameterSet: 78 parameters | contexts: dataset, figure, compute, constraint>

Available Backends
----------------------------

See the [Compute Tutorial](./compute) for details on adding compute options and using them to create synthetic models.

The following list in any version of PHOEBE can always be accessed via [phoebe.list_available_computes](../api/phoebe.list_available_computes.md).

Note also that all of these are listed on the [backends](../backends.md) page and their available functionality is compared in the [compute backend comparison table](../examples/compute_comparison_table.ipynb).

In [5]:
phoebe.list_available_computes()

['ellc', 'jktebop', 'legacy', 'phoebe']

### PHOEBE 1.0 Legacy

For more details, see [Comparing PHOEBE 2.0 vs PHOEBE Legacy](../examples/legacy) and the [legacy compute API docs](../api/phoebe.parameters.compute.legacy.md).

In [6]:
b.add_compute('legacy', compute='legacybackend')
print(b.get_compute('legacybackend'))

ParameterSet: 14 parameters
   sample_from@legacybackend@c...: []
   comments@legacybackend@compute: 
   pblum_method@legacybackend@...: phoebe
   irrad_method@legacybackend@...: wilson
   refl_num@legacybackend@compute: 1
         ie@legacybackend@compute: False
    enabled@legacybackend@compute: True
   atm@primary@legacybackend@c...: extern_atmx
   atm@secondary@legacybackend...: extern_atmx
   gridsize@primary@legacyback...: 60
   gridsize@secondary@legacyba...: 60
   distortion_method@primary@l...: roche
   distortion_method@secondary...: roche
   fti_method@legacybackend@co...: none


### ellc

For more details, see the the [ellc compute API docs](../api/phoebe.parameters.compute.ellc.md).

In [7]:
b.add_compute('ellc', compute='ellcbackend')
print(b.get_compute('ellcbackend'))

ParameterSet: 13 parameters
   sample_from@ellcbackend@com...: []
     comments@ellcbackend@compute: 
   pblum_method@ellcbackend@co...: stefan-boltzmann
   exact_grav@ellcbackend@compute: False
   irrad_method@ellcbackend@co...: lambert
      enabled@ellcbackend@compute: True
   atm@primary@ellcbackend@com...: ck2004
   atm@secondary@ellcbackend@c...: ck2004
   distortion_method@primary@e...: roche
   distortion_method@secondary...: roche
   grid@primary@ellcbackend@co...: default
   grid@secondary@ellcbackend@...: default
   fti_method@ellcbackend@compute: none


### jktebop

For more details, see the the [jktebop compute API docs](../api/phoebe.parameters.compute.jktebop.md).

In [8]:
b.add_compute('jktebop', compute='jktebopcompute')
print(b.get_compute('jktebopcompute'))

ParameterSet: 11 parameters
   sample_from@jktebopcompute@...: []
   comments@jktebopcompute@com...: 
   requiv_max_limit@jktebopcom...: 0.5
   pblum_method@jktebopcompute...: stefan-boltzmann
   ringsize@jktebopcompute@com...: 5.0 deg
   distortion_method@jktebopco...: sphere/biaxial spheroid
   irrad_method@jktebopcompute...: biaxial-spheroid
   enabled@jktebopcompute@compute: True
   atm@primary@jktebopcompute@...: ck2004
   atm@secondary@jktebopcomput...: ck2004
   fti_method@jktebopcompute@c...: none


Using Alternate Backends
---------------------------------


### Adding Compute Options

Adding a set of compute options, via [b.add_compute](../api/phoebe.frontend.bundle.Bundle.add_compute.md) for an alternate backend is just as easy as for the PHOEBE backend.  Simply provide the function or name of the function in [phoebe.parameters.compute](../api/phoebe.parameters.compute.md) that points to the parameters for that backend.

Here we'll add the default PHOEBE backend as well as the PHOEBE 1.0 (legacy) backend.  Note that in order to use an alternate backend, that backend must be installed on your machine.

In [9]:
b.add_compute('phoebe', compute='phoebebackend')

<ParameterSet: 35 parameters | datasets: lc01, _default, orb01>

In [10]:
print(b.get_compute('phoebebackend'))

ParameterSet: 19 parameters
   sample_from@phoebebackend@c...: []
   comments@phoebebackend@compute: 
   dynamics_method@phoebebacke...: keplerian
       ltte@phoebebackend@compute: False
   irrad_method@phoebebackend@...: horvat
   boosting_method@phoebebacke...: none
   eclipse_method@phoebebacken...: native
   horizon_method@phoebebacken...: boolean
   enabled@orb01@phoebebackend...: True
   enabled@lc01@phoebebackend@...: True
   mesh_method@primary@phoebeb...: marching
   mesh_method@secondary@phoeb...: marching
   ntriangles@primary@phoebeba...: 1500
   ntriangles@secondary@phoebe...: 1500
   distortion_method@primary@p...: roche
   distortion_method@secondary...: roche
   atm@primary@phoebebackend@c...: ck2004
   atm@secondary@phoebebackend...: ck2004
   fti_method@phoebebackend@co...: none


### Running Compute

Nothing changes when calling [b.run_compute](../api/phoebe.frontend.bundle.Bundle.run_compute.md) - simply provide the compute tag for those options.  Do note, however, that not all backends support all dataset types.

But, since the legacy backend doesn't support ck2004 atmospheres and interpolated limb-darkening, we do need to choose a limb-darkening law.  We can do this for all passband-component combinations by using [set_value_all](../api/phoebe.parameters.ParameterSet.set_value_all.md).

For more information on limb-darkening options, see the [limb-darkening tutorial](limb_darkening.ipynb).

In [11]:
b.set_value_all(qualifier='ld_mode', value='manual')

Wed, 24 Feb 2021 10:41 BUNDLE       WARNING ld_mode='interp' not supported by 'legacy' backend used by compute='legacybackend'.  Change ld_mode@primary@lc01.  If not addressed, this warning will continue to be raised and will throw an error at run_compute.
Wed, 24 Feb 2021 10:41 BUNDLE       WARNING ld_mode='interp' not supported by 'ellc' backend used by compute='ellcbackend'.  Change ld_mode@primary@lc01.  If not addressed, this warning will continue to be raised and will throw an error at run_compute.
Wed, 24 Feb 2021 10:41 BUNDLE       WARNING ld_mode='interp' not supported by 'jktebop' backend used by compute='jktebopcompute'.  Change ld_mode@primary@lc01.  If not addressed, this warning will continue to be raised and will throw an error at run_compute.
Wed, 24 Feb 2021 10:41 BUNDLE       WARNING ld_mode='interp' not supported by 'legacy' backend used by compute='legacybackend'.  Change ld_mode@secondary@lc01.  If not addressed, this warning will continue to be raised and will thr

In [12]:
b.set_value_all(qualifier='ld_func', value='logarithmic')

In [13]:
b.run_compute('legacybackend', model='legacyresults')

Wed, 24 Feb 2021 10:41 IO           WARNING requiv_max@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING logg@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING freq@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING pitch@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING yaw@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING long_an@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING irrad_frac_lost_bol@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING ld_mode_bol@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING ld_func_bol@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING ld_coeffs_source_bol@primary@star@co

<ParameterSet: 3 parameters | qualifiers: times, fluxes, comments>

Running Multiple Backends Simultaneously
---------------------------------------------------

Running multiple backends simultaneously is just as simple as running the PHOEBE backend with multiple sets of compute options (see [Compute](compute.html)).

We just need to make sure that each dataset is only enabled for one (or none) of the backends that we want to use, and then send a list of the compute tags to run_compute.  Here we'll use the PHOEBE backend to compute orbits and the legacy backend to compute light curves.

In [14]:
b.set_value_all(qualifier='enabled', dataset='lc01', compute='phoebebackend', value=False)
#b.set_value_all(qualifier='enabled', dataset='orb01', compute='legacybackend', value=False)  # don't need this since legacy NEVER computes orbits
print(b.filter(qualifier='enabled'))

ParameterSet: 7 parameters
   enabled@orb01@phoebe01@compute: True
    enabled@lc01@phoebe01@compute: True
    enabled@legacybackend@compute: True
      enabled@ellcbackend@compute: True
   enabled@jktebopcompute@compute: True
   enabled@orb01@phoebebackend...: True
   enabled@lc01@phoebebackend@...: False


In [15]:
b.run_compute(['phoebebackend', 'legacybackend'], model='mixedresults')

100%|██████████| 1000/1000 [00:00<00:00, 4767.99it/s]
Wed, 24 Feb 2021 10:41 IO           WARNING requiv_max@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING logg@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING freq@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING pitch@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING yaw@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING long_an@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING irrad_frac_lost_bol@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING ld_mode_bol@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO           WARNING ld_func_bol@primary@star@component has no phoebe 1 corollary
Wed, 24 Feb 2021 10:41 IO 

<ParameterSet: 18 parameters | kinds: orb, lc>

The parameters inside the returned model even remember which set of compute options (and therefore, in this case, which backend) were used to compute them.

In [16]:
print(b.filter(model='mixedresults').computes)

['phoebebackend', 'legacybackend']


In [17]:
b.filter(model='mixedresults', compute='phoebebackend').datasets

['orb01']

In [18]:
b.filter(model='mixedresults', compute='legacybackend').datasets

['lc01']